# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [1]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [2]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=1_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

(1000, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

#### Análise de Target

In [3]:
df['loan_status'].value_counts(dropna=False, normalize=True) * 100

loan_status
Fully Paid            72.8
Charged Off           14.5
Current               12.1
Late (31-120 days)     0.5
In Grace Period        0.1
Name: proportion, dtype: float64

In [4]:
TARGET = 'target_risco_credito'

status_de_risco = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    #"Late (16-30 days)"
]
df[TARGET] = df["loan_status"].isin(status_de_risco).astype(int)

df[TARGET].value_counts(dropna=False, normalize=True) * 100

target_risco_credito
0    85.0
1    15.0
Name: proportion, dtype: float64

### Safras

In [5]:
#date_col=['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d','next_pymnt_d']

temporal_columns = [
    "issue_d",                      # Data de emissão do empréstimo
    "earliest_cr_line",             # Primeira linha de crédito do cliente
    "last_pymnt_d",                 # Último pagamento realizado
    "last_credit_pull_d",           # Última consulta ao crédito
    "next_pymnt_d",                 # Próximo pagamento previsto (se aplicável)
    #"last_pymnt_amnt",             # (associada à data de pagamento)
    "debt_settlement_flag_date",    # Data em que houve acordo de dívida
    "settlement_date"               # Data em que o acordo foi fechado
]


# tratamento para colunas de data
for col in temporal_columns:
    df[col] = pd.to_datetime(df[col], format="%b-%Y")


df["safra"] = df["issue_d"].dt.to_period("M")#.astype(str)  # formato 'YYYY-MM'

print(df["safra"].min())
print(df["safra"].max())

2015-12
2015-12


### Captura Inicial

In [6]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    target_col=TARGET,
    default_keys=["id"]
)

trail.take_snapshot(df, name="amostra")

In [7]:
trail.describe_snapshot("amostra")


📄 Descrição do snapshot 'amostra':

▶️ Shape: (1000, 153)
▶️ Chaves de duplicação: ['id']
   • Duplicatas nas chaves: 0

🧱 Tipos de dados:


id                           int64
member_id                  float64
loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
                           ...    
settlement_amount          float64
settlement_percentage      float64
settlement_term            float64
target_risco_credito         int32
safra                    period[M]
Length: 153, dtype: object


🔎 Colunas detectadas automaticamente:
   • Numéricas (89): ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'char

member_id                                     1000
revol_bal_joint                               1000
sec_app_mths_since_last_major_derog           1000
sec_app_collections_12_mths_ex_med            1000
sec_app_chargeoff_within_12_mths              1000
sec_app_num_rev_accts                         1000
sec_app_open_act_il                           1000
sec_app_revol_util                            1000
sec_app_open_acc                              1000
sec_app_inq_last_6mths                        1000
sec_app_earliest_cr_line                      1000
sec_app_fico_range_high                       1000
sec_app_fico_range_low                        1000
sec_app_mort_acc                              1000
desc                                          1000
verification_status_joint                      992
annual_inc_joint                               992
dti_joint                                      992
orig_projected_additional_accrued_interest     991
payment_plan_start_date        


📊 Estatísticas numéricas:


,count,mean,std,min,25%,50%,75%,max
id,1000.0,6.836271e+07,392743.665605,65916558.00,68376699.50,6.846556e+07,6.853761e+07,68617057.00
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,1000.0,1.558220e+04,8694.531430,1000.00,9000.00,1.500000e+04,2.100000e+04,35000.00
funded_amnt,1000.0,1.558220e+04,8694.531430,1000.00,9000.00,1.500000e+04,2.100000e+04,35000.00
funded_amnt_inv,1000.0,1.558220e+04,8694.531430,1000.00,9000.00,1.500000e+04,2.100000e+04,35000.00
...,...,...,...,...,...,...,...,...
hardship_last_payment_amount,10.0,1.931880e+02,264.078880,0.71,18.58,7.279000e+01,2.367025e+02,756.87
settlement_amount,22.0,4.677728e+03,3424.722231,303.00,2737.00,4.163155e+03,6.122140e+03,13403.00
settlement_percentage,22.0,4.667545e+01,3.887082,44.93,45.00,4.500000e+01,4.502000e+01,60.01
settlement_term,22.0,1.209091e+01,8.366083,1.00,4.50,1.200000e+01,1.800000e+01,24.00



🏷️ Estatísticas categóricas:


,count,unique,top,freq
term,1000,2,36 months,684
grade,1000,7,B,292
sub_grade,1000,33,A1,70
emp_title,957,742,Teacher,37
emp_length,958,11,10+ years,359
home_ownership,1000,3,MORTGAGE,552
verification_status,1000,3,Not Verified,454
loan_status,1000,5,Fully Paid,728
pymnt_plan,1000,1,n,1000
url,1000,1000,https://lendingclub.com/browse/loanDetail.acti...,1



📈 Histogramas (categorias apenas):
  term: 2 valores distintos (top 3: {' 36 months': 684, ' 60 months': 316})
  grade: 7 valores distintos (top 3: {'B': 292, 'C': 272, 'A': 213})
  sub_grade: 33 valores distintos (top 3: {'A1': 70, 'C2': 68, 'B2': 65})
  emp_length: 12 valores distintos (top 3: {'10+ years': 359, '2 years': 102, '< 1 year': 89})
  home_ownership: 3 valores distintos (top 3: {'MORTGAGE': 552, 'RENT': 324, 'OWN': 124})
  verification_status: 3 valores distintos (top 3: {'Not Verified': 454, 'Source Verified': 330, 'Verified': 216})
  loan_status: 5 valores distintos (top 3: {'Fully Paid': 728, 'Charged Off': 145, 'Current': 121})
  pymnt_plan: 1 valores distintos (top 3: {'n': 1000})
  purpose: 11 valores distintos (top 3: {'debt_consolidation': 576, 'credit_card': 258, 'home_improvement': 59})
  title: 12 valores distintos (top 3: {'Debt consolidation': 559, 'Credit card refinancing': 249, 'Home improvement': 56})
  addr_state: 47 valores distintos (top 3: {'CA': 106,

#### Limpeza com Vassoura

In [8]:
assert set(df[TARGET].dropna().unique()) == {0, 1}, "TARGET não é binário!"

In [ ]:
# futuramente podemos pensar no Vassoura como um orquestrador, fazendo diferentes tipos de limpeza para a mesma heuristica
# assim: uma limpeza de correlação no início, um pouco mais 

vs = Vassoura(
    df,
    id_cols = ['id','member_id'],
    date_cols=['safra'],
    ignore_cols=['url']+temporal_columns,
    drop_ignored=True,
    target_col=TARGET,
    verbose='basic',
    
    engine='polars',
    
    adaptive_sampling=True,

    heuristics=[
        'missing',
        'variance',
        "iv",
        "graph_cut",
        "corr",
        "vif",
        'importance',
        'ks_separation',

        # corrigir implementação das heuristicas abaixo
        #"psi_stability",
        #"perm_importance_lgbm",
        #"partial_corr_cluster",
        #"drift_vs_target_leakage",
    ],
    
    params={
        "missing": 0.60,
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,
    },
    n_steps=2,         # controla quantas remoções por iteração
    vif_n_steps=2      # VIF será sempre 1-step (a menos que você altere)
)

df_limpo = vs.run(recompute=True)

TypeError: Vassoura.__init__() got an unexpected keyword argument 'thresholds'

In [ ]:
df_limpo.head()

In [ ]:
vs.generate_report("../reports/vassoura_report.html")